In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
from keras.models import Model, Input
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, UpSampling2D
from tensorflow.keras.callbacks import EarlyStopping
from keras.preprocessing import image
from sklearn.model_selection import train_test_split
import glob

**5.1 Data Preparation**

In [4]:
# List all filename in face image path
filenames = glob.glob('*.jpg')

# Load image files and Resize (h,w,ch) -> h = w < 100 , ch = 3 (R,G,B)

# Convert the image to an array

# Normalized color image


**5.2 ADD Noise**

In [ ]:
# Add Noise
#   Scala between 0-1
noise_factor = 0.1 

# กำหนด noise parameter


# Use np.random.normal to generate normal distribution (gaussian) noise

**5.3 Autoencoder Model**

In [ ]:
# กำหนด Encoder Architecture